# TopicGPT_Python package

`topicgpt_python` consists of five modules in total: 
- `generate_topic_lvl1` generates high-level and generalizable topics. 
- `generate_topic_lvl2` generates low-level and specific topics to each high-level topic.
- `refine_topics` refines the generated topics by merging similar topics and removing irrelevant topics.
- `assign_topics` assigns the generated topics to the input text, along with a quote that supports the assignment.
- `correct_topics` corrects the generated topics by reprompting the model so that the topic assignment is grounded in the topic list. 

![topicgpt_python](assets/img/pipeline.png)

## Setup
1. Make a new Python 3.9+ environment using virtualenv or conda. 
2. Install the required packages: `pip install --upgrade topicgpt_python`.
- Our package supports OpenAI API, Google Cloud Vertex AI API, Gemini API, Azure API, and vLLM inference. vLLM requires GPUs to run. 
- Please refer to https://openai.com/pricing/ for OpenAI API pricing or to https://cloud.google.com/vertex-ai/pricing for Vertex API pricing. 

In [1]:
# Run in shell
#!pip install --upgrade topicgpt_python

# Needed only for the OpenAI API deployment
#export OPENAI_API_KEY={your_openai_api_key}

# Needed only for the Vertex AI deployment
#export VERTEX_PROJECT={your_vertex_project}   # e.g. my-project
#export VERTEX_LOCATION={your_vertex_location} # e.g. us-central1

# Needed only for Gemini deployment
#export GEMINI_API_KEY={your_gemini_api_key}

# Needed only for the Azure API deployment
#export AZURE_OPENAI_API_KEY={your_azure_api_key}
#export AZURE_OPENAI_ENDPOINT={your_azure_endpoint}

## Usage
1. First, define the necessary file paths for I/O operations in `config.yml`. 
2. Then, import the necessary modules and functions from `topicgpt_python`.
3. Store your data in `data/input` and modify the `data_sample` path in `config.yml`. 

- Prepare your `.jsonl` data file in the following format:
    ```
    {
        "id": "IDs (optional)",
        "text": "Documents",
        "label": "Ground-truth labels (optional)"
    }
    ```

In [2]:
import os

os.environ['USE_LIBUV'] = '0'

In [3]:
from topicgpt_python import *
import yaml

with open("config.yml", "r") as f:
    config = yaml.safe_load(f)

INFO 02-12 22:33:51 __init__.py:192] Automatically detected platform cuda.


### Topic Generation 
Generate high-level topics using `generate_topic_lvl1`. 
- Define the api type and model. 
- Define your seed topics in `prompt/seed_1.md`.
- (Optional) Modify few-shot examples in `prompt/generation_1.txt`.
- Expect the generated topics in `data/output/{data_name}/generation_1.md` and `data/output/{data_name}/generation_1.jsonl`.
- Right now, early stopping is set to 100, meaning that if no new topic has been generated in the last 100 iterations, the generation process will stop.

In [4]:
#MODEL_API = 'vllm'
MODEL_API = 'custom_llm'

MODEL_NAME = "C:/git/Mistral-7B-Instruct-v0.3-quantized.w4a16"

generate_topic_lvl1(
    MODEL_API,
    MODEL_NAME,
    config["data_sample"],
    config["generation"]["prompt"],
    config["generation"]["seed"],
    config["generation"]["output"],
    config["generation"]["topic_output"],
    verbose=config["verbose"],
)

C:\Users\slick\.conda\envs\topic_gpt_gpu_updated\Lib\site-packages\auto_gptq\nn_modules\triton_utils\kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
C:\Users\slick\.conda\envs\topic_gpt_gpu_updated\Lib\site-packages\auto_gptq\nn_modules\triton_utils\kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
C:\Users\slick\.conda\envs\topic_gpt_gpu_updated\Lib\site-packages\auto_gptq\nn_modules\triton_utils\kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`

Preparing to import and load with custom_llm
Model loaded with custom_llm
Tokenizer loaded with custom_llm
-------------------
Initializing topic generation...
Model: C:/git/Mistral-7B-Instruct-v0.3-quantized.w4a16
Data file: data/input/sample.jsonl
Prompt file: prompt/generation_1.txt
Seed file: prompt/seed_1.md
Output file: data/output/sample/generation_1.jsonl
Topic file: data/output/sample/generation_1.md
-------------------


C:\Users\slick\.conda\envs\topic_gpt_gpu_updated\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


iterative_prompt system_message: You are a helpful assistant.
iterative_prompt prompt: You will receive a document and a set of top-level topics from a topic hierarchy. Your task is to identify generalizable topics within the document that can act as top-level topics in the hierarchy. If any relevant topics are missing from the provided set, please add them. Otherwise, output the existing top-level topics as identified in the document.

[Top-level topics]
[1] Trade

[Examples]
Example 1: Adding "[1] Agriculture"
Document: 
Saving Essential American Sailors Act or SEAS Act - Amends the Moving Ahead for Progress in the 21st Century Act (MAP-21) to repeal the Act's repeal of the agricultural export requirements that: (1) 25% of the gross tonnage of certain agricultural commodities or their products exported each fiscal year be transported on U.S. commercial vessels, and (2) the Secretary of Transportation (DOT) finance any increased ocean freight charges incurred in the transportation of 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

type(custom_llm_output): <class 'torch.Tensor'>
custom_llm_output: tensor([[    1,     3,  1763,  ...,  1265, 29491,     2]], device='cuda:0')
custom_llm_output_text: 1. Topic Level: 1
Topic Label: Oil and Gas Exploration
Topic Description: Policies and regulations related to the exploration, production, and decommissioning of offshore oil and gas platforms, including their impact on marine habitats and fisheries.</s>
Invalid topic format: 1. Topic Level: 1. Skipping...
Invalid topic format: Topic Label: Oil and Gas Exploration. Skipping...
Invalid topic format: Topic Description: Policies and regulations related to the exploration, production, and decommissioning of offshore oil and gas platforms, including their impact on marine habitats and fisheries.</s>. Skipping...
Topics: 1. Topic Level: 1
Topic Label: Oil and Gas Exploration
Topic Description: Policies and regulations related to the exploration, production, and decommissioning of offshore oil and gas platforms, including their 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


type(custom_llm_output): <class 'torch.Tensor'>
custom_llm_output: tensor([[    1,     3,  1763,  1390,  6324,  1032,  4016,  1072,  1032,  1576,
          1070,  2598, 29501,  5172, 14585,  1245,  1032,  9835, 26614, 29491,
          4372,  4406,  1117,  1066,  9819,  3720, 12320, 14585,  3141,  1040,
          4016,  1137,  1309,  1728,  1158,  2598, 29501,  5172, 14585,  1065,
          1040, 26614, 29491,  1815,  1475,  9366, 14585,  1228,  7693,  1245,
          1040,  4625,  1576, 29493,  5433,  1735,  1474, 29491, 16278, 29493,
          4593,  1040,  7362,  2598, 29501,  5172, 14585,  1158, 11016,  1065,
          1040,  4016, 29491,   781,   781, 29560,  6996, 29501,  5172, 14585,
         29561,   781, 29560, 29508, 29561, 18452,   781,   781, 29560,  1734,
         10642, 29561,   781, 21043, 29473, 29508, 29515,  4069,  1056, 15032,
         29508, 29561, 24605,  1250, 29507,   781,  8132, 29515, 29473,   781,
         29503,  2420, 12067,  3024,  3324,  1086,  1382,  1502,

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [04:35<00:00, 55.11s/it]

type(custom_llm_output): <class 'torch.Tensor'>
custom_llm_output: tensor([[    1,     3,  1763,  1390,  6324,  1032,  4016,  1072,  1032,  1576,
          1070,  2598, 29501,  5172, 14585,  1245,  1032,  9835, 26614, 29491,
          4372,  4406,  1117,  1066,  9819,  3720, 12320, 14585,  3141,  1040,
          4016,  1137,  1309,  1728,  1158,  2598, 29501,  5172, 14585,  1065,
          1040, 26614, 29491,  1815,  1475,  9366, 14585,  1228,  7693,  1245,
          1040,  4625,  1576, 29493,  5433,  1735,  1474, 29491, 16278, 29493,
          4593,  1040,  7362,  2598, 29501,  5172, 14585,  1158, 11016,  1065,
          1040,  4016, 29491,   781,   781, 29560,  6996, 29501,  5172, 14585,
         29561,   781, 29560, 29508, 29561, 18452,   781,   781, 29560,  1734,
         10642, 29561,   781, 21043, 29473, 29508, 29515,  4069,  1056, 15032,
         29508, 29561, 24605,  1250, 29507,   781,  8132, 29515, 29473,   781,
         29503,  2420, 12067,  3024,  3324,  1086,  1382,  1502,

### Topic Refinement
If topics are generated by a weaker model, there sometimes exist irrelevant or redundant topics. This module: 
- Merges similar topics.
- Removes overly specific or redundant topics that occur < 1% of the time (you can skip this by setting `remove` to False in `config.yml`).
- Expect the refined topics in `data/output/{data_name}/refinement_1.md` and `data/output/{data_name}/refinement_1.jsonl`. If nothing happens, it means that the topic list is coherent.
- If you're unsatisfied with the refined topics, call the function again with the refined topic file and refined topic file from the previous iteration

In [5]:
# Optional: Refine topics if needed
if config["refining_topics"]:
    refine_topics(
        MODEL_API,
        MODEL_NAME,
        config["refinement"]["prompt"],
        config["generation"]["output"],
        config["generation"]["topic_output"],
        config["refinement"]["topic_output"],
        config["refinement"]["output"],
        verbose=config["verbose"],
        remove=config["refinement"]["remove"],
        mapping_file=config["refinement"]["mapping_file"]
    )

Preparing to import and load with custom_llm
Model loaded with custom_llm
Tokenizer loaded with custom_llm
-------------------
Initializing topic refinement...
Model: C:/git/Mistral-7B-Instruct-v0.3-quantized.w4a16
Input data file: data/output/sample/generation_1.jsonl
Prompt file: prompt/refinement.txt
Output file: data/output/sample/refinement.md
Topic file: data/output/sample/generation_1.md
-------------------
No topic pairs to be merged.
No topics removed.
Node('/Topics', count=1, desc='Root topic', lvl=0)


### Subtopic Generation 
Generate subtopics using `generate_topic_lvl2`.
- This function iterates over each high-level topic and generates subtopics based on a few example documents associated with the high-level topic.
- Expect the generated topics in `data/output/{data_name}/generation_2.md` and `data/output/{data_name}/generation_2.jsonl`.

In [6]:
# Optional: Generate subtopics
if config["generate_subtopics"]:
    generate_topic_lvl2(
        MODEL_API,
        MODEL_NAME,
        config["generation"]["topic_output"],
        config["generation"]["output"],
        config["generation_2"]["prompt"],
        config["generation_2"]["output"],
        config["generation_2"]["topic_output"],
        verbose=config["verbose"],
    )

Preparing to import and load with custom_llm
Model loaded with custom_llm
Tokenizer loaded with custom_llm
-------------------
Initializing topic generation (lvl 2)...
Model: C:/git/Mistral-7B-Instruct-v0.3-quantized.w4a16
Data file: data/output/sample/generation_1.jsonl
Prompt file: prompt/generation_2.txt
Seed file: data/output/sample/generation_1.md
Output file: data/output/sample/generation_2.jsonl
Topic file: data/output/sample/generation_2.md
-------------------
Number of remaining documents for prompting: 0


0it [00:00, ?it/s]


### Topic Assignment
Assign the generated topics to the input text using `assign_topics`. Each assignment is supported by a quote from the input text.
- Expect the assigned topics in `data/output/{data_name}/assignment.jsonl`. 
- The model used here is often a weaker model to save cost, so the topics may not be grounded in the topic list. To correct this, use the `correct_topics` module. If there are still errors/hallucinations, run the `correct_topics` module again.

In [7]:
# Assignment
assign_topics(
    MODEL_API,
    MODEL_NAME,
    config["data_sample"],
    config["assignment"]["prompt"],
    config["assignment"]["output"],
    config["generation"][
        "topic_output"
    ],  # TODO: change to generation_2 if you have subtopics, or config['refinement']['topic_output'] if you refined topics
    verbose=config["verbose"],
)

Preparing to import and load with custom_llm
Model loaded with custom_llm
Tokenizer loaded with custom_llm
-------------------
Initializing topic assignment...
Model: C:/git/Mistral-7B-Instruct-v0.3-quantized.w4a16
Data file: data/input/sample.jsonl
Prompt file: prompt/assignment.txt
Output file: data/output/sample/assignment.jsonl
Topic file: data/output/sample/generation_1.md
-------------------


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 59.46it/s]

batch_prompt prompts: ['You will receive a document and a topic hierarchy. Assign the document to the most relevant topics the hierarchy. Then, output the topic labels, assignment reasoning and supporting quotes from the document. DO NOT make up new topics or quotes.  \n\n[Topic Hierarchy]\n\n\n[Examples]\nExample 1: Assign "[1] Agriculture" to the document\nDocument: \nSaving Essential American Sailors Act or SEAS Act - Amends the Moving Ahead for Progress in the 21st Century Act (MAP-21) to repeal the Act\'s repeal of the agricultural export requirements that: (1) 25% of the gross tonnage of certain agricultural commodities or their products exported each fiscal year be transported on U.S. commercial vessels, and (2) the Secretary of Transportation (DOT) finance any increased ocean freight charges incurred in the transportation of such items.\n\nAssignment:\n[1] Agriculture: Mentions changes in agricultural export requirements ("...repeal of the agricultural export requirements that.

TypeError: can only concatenate str (not "bool") to str

In [ ]:
# Correction
correct_topics(
    MODEL_API,
    MODEL_NAME,
    config["assignment"]["output"],
    config["correction"]["prompt"],
    config["generation"][
        "topic_output"
    ],  # TODO: change to generation_2 if you have subtopics, or config['refinement']['topic_output'] if you refined topics
    config["correction"]["output"],
    verbose=config["verbose"],
)